In [1]:
import cv2
import numpy as np

In [2]:
# 1. 读取视频和设置参数
video_path = "../lhx/piper_real_cnn/data/20250808/seq_115654_color.mp4"

In [3]:
import cv2
import numpy as np
import os
from pathlib import Path

def apply_grayscale(frame):
    """转换为灰度图"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # 转回3通道以便保存为彩色视频
        return cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    return frame

def apply_canny_edge(frame):
    """Canny边缘检测"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 应用高斯模糊减少噪声
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Canny边缘检测
    edges = cv2.Canny(blurred, 50, 150)
    
    # 转回3通道
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)

def apply_sobel_edge(frame):
    """Sobel边缘检测"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # Sobel X和Y方向的梯度
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    # 计算梯度幅度
    sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)
    sobel_combined = np.uint8(sobel_combined / sobel_combined.max() * 255)
    
    return cv2.cvtColor(sobel_combined, cv2.COLOR_GRAY2BGR)

def apply_laplacian_edge(frame):
    """拉普拉斯边缘检测"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 应用高斯模糊
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    
    # 拉普拉斯算子
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    laplacian = np.uint8(np.absolute(laplacian))
    
    return cv2.cvtColor(laplacian, cv2.COLOR_GRAY2BGR)

def apply_contours(frame):
    """轮廓检测"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 二值化
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 找到轮廓
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 创建空白图像绘制轮廓
    contour_img = np.zeros_like(frame)
    cv2.drawContours(contour_img, contours, -1, (255, 255, 255), 2)
    
    return contour_img

def apply_morphology(frame):
    """形态学操作 - 突出形状特征"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 形态学梯度 (膨胀 - 腐蚀)
    kernel = np.ones((3, 3), np.uint8)
    morph_gradient = cv2.morphologyEx(gray, cv2.MORPH_GRADIENT, kernel)
    
    return cv2.cvtColor(morph_gradient, cv2.COLOR_GRAY2BGR)

def apply_adaptive_threshold(frame):
    """自适应阈值 - 保留局部特征"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 自适应阈值
    adaptive_thresh = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    
    return cv2.cvtColor(adaptive_thresh, cv2.COLOR_GRAY2BGR)

def apply_skeleton(frame):
    """骨架化 - 提取形状骨架"""
    if len(frame.shape) == 3:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        gray = frame
    
    # 二值化
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # 骨架化
    skeleton = np.zeros(binary.shape, np.uint8)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    
    while True:
        eroded = cv2.erode(binary, kernel)
        dilated = cv2.dilate(eroded, kernel)
        diff = cv2.subtract(binary, dilated)
        skeleton = cv2.bitwise_or(skeleton, diff)
        binary = eroded.copy()
        
        if cv2.countNonZero(binary) == 0:
            break
    
    return cv2.cvtColor(skeleton, cv2.COLOR_GRAY2BGR)

def process_video_with_shape_methods(input_video_path, output_dir):
    """
    使用多种方法处理视频，去除颜色特征，保留形状特征
    """
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 定义处理方法
    methods = {
        'grayscale': apply_grayscale,
        'canny_edge': apply_canny_edge,
        'sobel_edge': apply_sobel_edge,
        'laplacian_edge': apply_laplacian_edge,
        'contours': apply_contours,
        'morphology': apply_morphology,
        'adaptive_threshold': apply_adaptive_threshold,
        'skeleton': apply_skeleton
    }
    
    # 获取输入视频的基本信息
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video {input_video_path}")
        return
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Processing video: {input_video_path}")
    print(f"Resolution: {width}x{height}, FPS: {fps}, Total frames: {total_frames}")
    
    # 为每种方法创建视频写入器
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writers = {}
    
    input_filename = Path(input_video_path).stem
    
    for method_name in methods.keys():
        output_path = os.path.join(output_dir, f"{input_filename}_{method_name}.mp4")
        writers[method_name] = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 对每种方法应用处理
        for method_name, method_func in methods.items():
            try:
                processed_frame = method_func(frame.copy())
                writers[method_name].write(processed_frame)
            except Exception as e:
                print(f"Error processing frame {frame_count} with method {method_name}: {e}")
        
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")
    
    # 释放资源
    cap.release()
    for writer in writers.values():
        writer.release()
    
    print(f"Processing complete! Output videos saved to: {output_dir}")
    
    # 打印输出文件列表
    print("\nGenerated videos:")
    for method_name in methods.keys():
        output_path = os.path.join(output_dir, f"{input_filename}_{method_name}.mp4")
        if os.path.exists(output_path):
            file_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
            print(f"  - {method_name}: {output_path} ({file_size:.1f} MB)")

def main():
    # 输入视频路径
    input_video = "../lhx/piper_real_cnn/data/20250808/seq_115654_color.mp4"
    
    # 输出目录
    output_directory = "shape_feature_videos"
    
    # 检查输入文件是否存在
    if not os.path.exists(input_video):
        print(f"Error: Input video not found: {input_video}")
        return
    
    # 处理视频
    process_video_with_shape_methods(input_video, output_directory)

if __name__ == "__main__":
    main()

Processing video: ../lhx/piper_real_cnn/data/20250808/seq_115654_color.mp4
Resolution: 640x360, FPS: 30, Total frames: 471
Processed 100/471 frames
Processed 200/471 frames
Processed 300/471 frames
Processed 400/471 frames
Processing complete! Output videos saved to: shape_feature_videos

Generated videos:
  - grayscale: shape_feature_videos/seq_115654_color_grayscale.mp4 (6.6 MB)
  - canny_edge: shape_feature_videos/seq_115654_color_canny_edge.mp4 (22.6 MB)
  - sobel_edge: shape_feature_videos/seq_115654_color_sobel_edge.mp4 (7.7 MB)
  - laplacian_edge: shape_feature_videos/seq_115654_color_laplacian_edge.mp4 (2.7 MB)
  - contours: shape_feature_videos/seq_115654_color_contours.mp4 (12.6 MB)
  - morphology: shape_feature_videos/seq_115654_color_morphology.mp4 (7.2 MB)
  - adaptive_threshold: shape_feature_videos/seq_115654_color_adaptive_threshold.mp4 (26.2 MB)
  - skeleton: shape_feature_videos/seq_115654_color_skeleton.mp4 (13.6 MB)
